## Setting every prediction probabilities as 0.5 on train set,  get score 0.6931471805599455.   
If this submission get close score on public leadboard, it suggest that the function calc_score() is logically right.  

Another check see  https://www.kaggle.com/kingstying/mean-baseline?scriptVersionId=43804369  version 2  

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
from sklearn.metrics import log_loss
import numpy as np
import pandas as pd

def get_train_qi():
    df = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
    qi = df[["pe_present_on_image","StudyInstanceUID"]].groupby("StudyInstanceUID").mean().reset_index()
    qi.columns = ["StudyInstanceUID", "qi"]
    return qi

def cross_entropy(targets, predictions,  epsilon=1e-12, reduction='none'):
    """
    This function is from https://www.kaggle.com/khyeh0719/0929-updated-rsna-competition-metric  
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k1, k2, ...) ndarray
           targets (N, k1, k2, ...) ndarray
           reduction: 'none' | 'mean' | 'sum'
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    
    ce = -(targets*np.log(predictions) + (1.-targets)*np.log(1.-predictions))
    
    if reduction == 'none':
        return ce
    
    ce = np.sum(ce)
    if reduction == 'sum':
        return ce
    
    if reduction == 'mean':
        ce /= predictions.shape[0]
        return ce

    assert False, "reduction should be 'none' | 'mean' | 'sum'".format(reduction)

def calc_score(df, df_probs, qi):
    cols_label = ["pe_present_on_image", \
              "negative_exam_for_pe", "indeterminate", "chronic_pe", \
              "acute_and_chronic_pe", "central_pe", "leftsided_pe", \
              "rightsided_pe", "rv_lv_ratio_gte_1", "rv_lv_ratio_lt_1", ]
    
    weights = [0.07361963  ,\
                            0.0736196319, 0.09202453988, 0.1042944785,\
                           0.1042944785, 0.1877300613, 0.06257668712, \
                           0.06257668712, 0.2346625767, 0.0782208589]
    
    assert (df["SOPInstanceUID"]==df_probs["SOPInstanceUID"]).all(), f"SOPInstanceUID not match!"
    
    target_exam = df[["StudyInstanceUID"] + cols_label[1:]].groupby("StudyInstanceUID").mean()
    probs_exam = df_probs[["StudyInstanceUID"] + cols_label[1:]].groupby("StudyInstanceUID").mean()
#     probs_exam = pd.merge(target_exam[["StudyInstanceUID"]], probs_exam,on="StudyInstanceUID", how="inner")
#     assert probs_exam.shape[0]==target_exam.shape[0], f"probs_exam.shape[0]!=target_exam.shape[0], {probs_exam.shape[0]}!={target_exam.shape[0]}"
    
    
    score_exam = []
    epsilon=1e-12
    for col,w in zip(cols_label[1:],weights[1:]):
        score = log_loss(target_exam[col].values, np.clip(probs_exam[col].values, epsilon, 1. - epsilon)    ) * w 
        score = score * target_exam.shape[0]   # calc sum, not mean
        score_exam.append(score)
        
    score_exam = np.sum(score_exam)  # sum, not mean
    
    df_probs = pd.merge(df_probs, qi, on="StudyInstanceUID", how="inner")
    df_probs["target-pe_present_on_image"]=df["pe_present_on_image"]
    

    df_probs["score_img"] = \
    df_probs[["target-pe_present_on_image","pe_present_on_image","qi"]].apply(lambda x:cross_entropy(x[0],x[1])*x[2]*weights[0], axis=1 )
    
    score_img = df_probs["score_img"].sum()
    
    total_score = score_exam + score_img
    total_weights = np.sum(weights[1:])*df.StudyInstanceUID.nunique() +  np.sum(weights[0]*df_probs["qi"].values)
    
    return total_score/total_weights

In [ ]:
sub = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv")

In [ ]:
sub.describe()

In [ ]:
cols_label = ["pe_present_on_image", \
              "negative_exam_for_pe", "indeterminate", "chronic_pe", \
              "acute_and_chronic_pe", "central_pe", "leftsided_pe", \
              "rightsided_pe", "rv_lv_ratio_gte_1", "rv_lv_ratio_lt_1", ]

In [ ]:
train = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
for c in cols_label:
    train[c] = 0.5

train_target = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")

qi = get_train_qi()
score = calc_score(train_target , train , qi)

print(f"score on train set {score}")

In [ ]:
sub.to_csv("submission.csv", index=False)